In [1]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# The Evidence Soft Halving Algorithm (ESH)
The ESH class implements a learning algorithm with multiple SGDClassifier experts. Initially, it assigns equal mass to each expert. It makes aggregated predictions based on experts' outputs. If the coarsened prediction results in a error, the experts' masses are updated according to their prediction accuracy using the Shafer-Dempster rule (a refined mass of alpha is assigned to the set of experts who were correct, while those who were incorrect receive mass of 1-alpha). No updates are made when all experts agree, as the situation is uninformative. The goal is to aggregate predictions in adversarial settings and minimize errors through dynamic mass updates.

In [2]:
class ESH:
    def __init__(self, num_experts, alpha, epsilon):
        self.total_errors = 0
        self.num_experts = num_experts
        self.alpha = alpha
        self.epsilon = epsilon
        self.experts_info = {}  # Dictionary to store expert information


    def initialize_experts(self, X_train, y_train):
        """
        Initializes experts as SGDClassifier models.
        Each expert is trained on a subset of the training data.
        """
        for i in range(self.num_experts):
            X_subtrain, _, y_subtrain, _ = train_test_split(X_train, y_train, test_size=0.5)
            clf = SGDClassifier(max_iter=1000, tol=1e-3)
            clf.fit(X_subtrain, y_subtrain)

            # Store model information, initial mass, and empty predictions list for each expert
            self.experts_info[i] = {
                'model': clf,
                'predictions': [],
                'mass': 1.0 / self.num_experts
            }

        print(f"Initialization: {self.num_experts} experts with masses {[info['mass'] for info in self.experts_info.values()]}")

    def coarsening(self, t):
        """
        Aggregates the predictions of active experts and calculates the coarsened prediction.
        Returns the coarsened masses for predictions of 1 and 0.
        """

        # Identify experts predicting 1 and 0 at time t
        experts_predicted_1 = {i for i, info in self.experts_info.items() if info['predictions'][t] == 1}
        experts_predicted_0 = {i for i, info in self.experts_info.items() if info['predictions'][t] == 0}

        print(f"Experts predicting 1: {experts_predicted_1}")
        print(f"Experts predicting 0: {experts_predicted_0}")

        # Compute the coarsened mass function
        mass_1 = np.sum([info['mass'] for i, info in self.experts_info.items() if info['mass'] > 0 and info['predictions'][t] == 1])
        mass_0 = np.sum([info['mass'] for i, info in self.experts_info.items() if info['mass'] > 0 and info['predictions'][t] == 0])

        print(f"Mass_1: {mass_1}, Mass_0: {mass_0}")

        return mass_1, mass_0

    def update_masses(self, t, y_true, epsilon):
        """
        Updates the experts' masses based on their predictions.
        Experts with zero mass are removed and the remaining masses are normalized if the total mass is grater than 1 + epsilon.
        Returns True if only one expert remains, otherwise returns False.
        """

        K = 0

        # Calculate the degree of conflict K
        for i in self.experts_info.keys():
            if self.experts_info[i]['predictions'][t] != y_true:
                K += self.alpha * self.experts_info[i]['mass']
            else:
                K += (1 - self.alpha) * self.experts_info[i]['mass']

        print(f"Degree of conflict K: {K}")

        # Compute the combined mass
        for i in list(self.experts_info.keys()):
            if self.experts_info[i]['predictions'][t] != y_true:
                self.experts_info[i]['mass'] *= (1 - self.alpha) / (1 - K)
            else:
                self.experts_info[i]['mass'] *= self.alpha / (1 - K)

        total_mass = np.sum([info['mass'] for info in self.experts_info.values()])
        print(f"Sum of the masses after the update: {total_mass}")

        # Check if total mass is near 1 and if not, normalize
        if total_mass > epsilon:
            for i in self.experts_info.keys():
                self.experts_info[i]['mass'] /= total_mass

        print(f"Updated masses: {[info['mass'] for info in self.experts_info.values()]}")

        # Remove experts with zero mass
        self.experts_info = {i: info for i, info in self.experts_info.items() if info['mass'] > 0}

        # Check if only one expert remains
        if len(self.experts_info) == 1:
            print("Just one remaining expert. Stopping the algorithm.")
            return True

        return False

    def run(self, X_test):
        """
        Runs the algorithm on the test data.
        """

        t = 0

        while t < len(X_test):

            print(f"\nRound: {t + 1}")

            # Collect predictions from each expert
            for i in self.experts_info:
                prediction = self.experts_info[i]['model'].predict(X_test[t].reshape(1, -1))[0]
                self.experts_info[i]['predictions'].append(prediction)

            predictions_list = [self.experts_info[i]['predictions'][t] for i in self.experts_info]
            print(f"Experts predictions for round {t + 1}: {predictions_list}")

            # Generate the adversarial label
            y_true = 1 - (np.bincount(predictions_list).argmax())
            print(f"Adversarial true label for round {t + 1}: {y_true}")

            # Check if all the predictions are the same
            all_equal = all(pred == predictions_list[0] for pred in predictions_list)

            # Coarsening
            print("\nCoarsening:")
            mass_1, mass_0 = self.coarsening(t)

            # Determine coarsened prediction
            aggregated_prediction = 1 if mass_1 >= mass_0 else 0
            print(f"Aggregated prediction: {aggregated_prediction}")

            if aggregated_prediction != y_true:
              print("The aggregated prediction is different from the true label.")
              self.total_errors += 1
              print(f"Update total errors: {self.total_errors}")

              # If experts agree completely, is not informative -> skip mass updating
              if all_equal:
                print("All experts agree completely. Skipping mass updating.")
                t += 1
                continue
              else:
                # If Error occurs, update the masses
                print("\nMass updating:")
                stop = self.update_masses(t, y_true, self.epsilon)
                if stop:
                  break
            else:
              print("The aggregated prediction is the same as the true label. No update is needed.")

            t += 1

        ########################################################################
        print()
        print("\nFinal Results:")
        print(f"\nTotal number of instances: {len(X_test)}")
        print(f"Remaining experts: {self.experts_info}")
        print(f"Total number of errors: {self.total_errors}")

In [3]:
# Test the algorithm
X, y = make_classification(n_samples=100, n_features=20, n_informative=15, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

num_experts = 3
alpha=0.8
epsilon=1e-10

esh = ESH(num_experts=num_experts, alpha=alpha, epsilon=epsilon)
esh.initialize_experts(X_train, y_train)
esh.run(X_test)

Initialization: 3 experts with masses [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

Round: 1
Experts predictions for round 1: [0, 1, 1]
Adversarial true label for round 1: 0

Coarsening:
Experts predicting 1: {1, 2}
Experts predicting 0: {0}
Mass_1: 0.6666666666666666, Mass_0: 0.3333333333333333
Aggregated prediction: 1
The aggregated prediction is different from the true label.
Update total errors: 1

Mass updating:
Degree of conflict K: 0.6
Sum of the masses after the update: 0.9999999999999999
Updated masses: [0.6666666666666667, 0.16666666666666666, 0.16666666666666666]

Round: 2
Experts predictions for round 2: [0, 0, 0]
Adversarial true label for round 2: 1

Coarsening:
Experts predicting 1: set()
Experts predicting 0: {0, 1, 2}
Mass_1: 0.0, Mass_0: 1.0
Aggregated prediction: 0
The aggregated prediction is different from the true label.
Update total errors: 2
All experts agree completely. Skipping mass updating.

Round: 3
Experts predictions for round 3: [0, 0, 1]